In [ ]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 13})

from scipy.ndimage.filters import gaussian_filter

from astropy.io import fits
from astropy.visualization import ZScaleInterval, AsinhStretch

from grism.eiger import get_wav_array

In [ ]:
### LOAD SPECTRA
# Stacked COLA1
path_to_spec = '../../spectra/stacked_2D_COLA1_9269.fits'
hdul = fits.open(path_to_spec)
hdul.info()

stretch = AsinhStretch(a=0.1)

emline_stack = stretch(hdul['EMLINE'].data)
emline_A = stretch(hdul['EMLINEA'].data)
emline_B = stretch(hdul['EMLINEB'].data)

wav_Arr = get_wav_array(path_to_spec)
z_COLA1 = 6.59161
wav_0_Arr = wav_Arr / (1 + z_COLA1)

aux = np.arange(emline_stack.shape[0])
mask_spec_height = (aux > 10) & (aux < 40)

emline_stack = emline_stack[mask_spec_height]
emline_A = emline_A[mask_spec_height]
emline_B = emline_B[mask_spec_height]

# Apply Gaussian smoothing to images
sigma = 0.6
emline_stack = gaussian_filter(emline_stack, sigma)
emline_A = gaussian_filter(emline_A, sigma)
emline_B = gaussian_filter(emline_B, sigma)

# The cutoff image
img = mpimg.imread('../../images/COLA1_color_cutoff.png')

In [ ]:
# Define limits for the plots
mask_Hgamma_wav = (wav_0_Arr > 4320) & (wav_0_Arr < 4390)
mask_O3_wav = (wav_0_Arr > 4840) & (wav_0_Arr < 5050)

spec_width_ratio = sum(mask_O3_wav) / sum(mask_Hgamma_wav)
spec_im_ratio = img.shape[0] / sum(mask_spec_height)

In [ ]:
fig = plt.figure(figsize=(14, 4.5))

# In the first row, the stacked spectra
gs = fig.add_gridspec(3, 4, width_ratios=[1, spec_width_ratio, 0.05, spec_im_ratio * 0.4],
                      height_ratios=[1, 1, 1])
ax00 = fig.add_subplot(gs[0, 0])
ax01 = fig.add_subplot(gs[0, 1])
ax10 = fig.add_subplot(gs[1, 0])
ax11 = fig.add_subplot(gs[1, 1])
ax20 = fig.add_subplot(gs[2, 0])
ax21 = fig.add_subplot(gs[2, 1])

axim = fig.add_subplot(gs[:, 3])

# vmax = emline_stack[:, mask_Hgamma_wav].max()
# vmin = emline_stack[:, mask_Hgamma_wav].min()
[vmin, vmax] = ZScaleInterval(contrast=0.3).get_limits(emline_stack.flatten())

ax20.imshow(emline_stack[:, mask_Hgamma_wav], vmax=vmax, vmin=vmin, rasterized=True)
ax21.imshow(emline_stack[:, mask_O3_wav], vmax=vmax, vmin=vmin, rasterized=True)

ax00.imshow(emline_A[:, mask_Hgamma_wav], vmax=vmax, vmin=vmin, rasterized=True)
ax01.imshow(emline_A[:, mask_O3_wav], vmax=vmax, vmin=vmin, rasterized=True)

ax10.imshow(emline_B[:, mask_Hgamma_wav], vmax=vmax, vmin=vmin, rasterized=True)
ax11.imshow(emline_B[:, mask_O3_wav], vmax=vmax, vmin=vmin, rasterized=True)

# Remove ax ticks
for ax in [ax00, ax01, ax10, ax11]:
    ax.set_xticks([])
    ax.set_xticklabels([])
for ax in [ax00, ax01, ax10, ax11, ax20, ax21]:
    ax.set_yticks([])
    ax.set_yticklabels([])

# Mark the lines
line_w0_list = [4960.295, 5008.240, 4862.68]
line_x_list = [np.argmin(np.abs(wav_0_Arr[mask_O3_wav] - xtw0)) for xtw0 in line_w0_list]
for ax in [ax01, ax11, ax21]:
    ax.vlines(x=line_x_list, ymax=sum(mask_spec_height) - 0.5, ymin=22,
              ls='--', color='r', lw=2)
 
line_w0_list = [4364.436, 4341.68]
line_x_list = [np.argmin(np.abs(wav_0_Arr[mask_Hgamma_wav] - xtw0)) for xtw0 in line_w0_list]
for ax in [ax00, ax10, ax20]:
    ax.vlines(x=line_x_list, ymax=sum(mask_spec_height) - 0.5, ymin=22,
              ls='--', color='r', lw=2)

# Set wavelength ticks
xticks_w0 = [4325, 4350, 4375]
xticks = [np.argmin(np.abs(wav_0_Arr[mask_Hgamma_wav] - xtw0)) for xtw0 in xticks_w0]
ax20.set_xticks(xticks)
ax20.set_xticklabels(xticks_w0)

xticks_w0 = np.arange(4850, 5050, 25)
xticks = [np.argmin(np.abs(wav_0_Arr[mask_O3_wav] - xtw0)) for xtw0 in xticks_w0]
ax21.set_xticks(xticks)
ax21.set_xticklabels(xticks_w0)

# Mark the image
# Angular pixel size = 0.03"
axim.vlines(x=img.shape[0] // 2 - 1, ymax=img.shape[1], ymin=80, color='white', ls='--')
axim.hlines(y=img.shape[1] // 2 - 1, xmax=img.shape[1], xmin=80, color='white', ls='--')

central_x = img.shape[0] // 2 - 1
axim.set_xticks([central_x - 33, central_x, central_x + 33])
# axim.set_xticklabels([r'-1$^{\prime\prime}$', r'0$^{\prime\prime}$', r'1$^{\prime\prime}$'])
axim.set_xticklabels([-1, 0, 1])

# Axes labels
ax21.set_xlabel(r'Rest-frame wavelength [\AA]', fontsize=15)

ax20.set_ylabel(r'\bf Combined', fontsize=14)
ax00.set_ylabel(r'\bf Module A', fontsize=14)
ax10.set_ylabel(r'\bf Module B', fontsize=14)

axim.imshow(img, rasterized=True)

axim.set_yticks([])
axim.set_yticklabels([])

axim.set_xlabel('Angular separation [arcsec]', fontsize=15)

gs.update(wspace=0.01, hspace=0.05)


# Add text with the line names
line_dict1 = {
    r'[OIII]$_{\lambda 4960}$': 4960.295,
    r'[OIII]$_{\lambda 5008}$': 5008.240,
    r'H$\beta$': 4862.68,
}
line_dict0 = {
    r'[OIII]$_{\lambda 4364}$': 4364.436,
    r'H$\gamma$': 4341.68
}
for linename, wavelength in line_dict1.items():
    wav_idx = np.argmin(np.abs(wav_0_Arr[mask_O3_wav] - wavelength))
    ax01.text(wav_idx, -1, linename, fontsize=13,
              horizontalalignment='center', verticalalignment='bottom')
for linename, wavelength in line_dict0.items():
    wav_idx = np.argmin(np.abs(wav_0_Arr[mask_Hgamma_wav] - wavelength))
    ax00.text(wav_idx, -1, linename, fontsize=13,
              horizontalalignment='center', verticalalignment='bottom')



savefig_path = '/home/alberto/cosmos/ista/COLA1/paper/figures'
fig.savefig(f'{savefig_path}/COLA1_2D_spec.pdf', bbox_inches='tight', pad_inches=0.1,
            facecolor='w')
            # facecolor='w', dpi=200)
plt.show()